In [65]:
import boto3
import pandas as pd
from io import StringIO # in-mem buffer we can use
from datetime import datetime, timedelta

In [66]:
# we want to build the report based on this starting date - 1
arg_date = '2021-5-07'
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)
arg_date_dt

datetime.date(2021, 5, 6)

In [67]:
# create s3 resource
s3 = boto3.resource("s3")
bucket = s3.Bucket('deutsche-boerse-xetra-pds')

In [68]:
# get list of all files on specific date - every hour of the day
# bucket_obj1 = bucket.objects.filter(Prefix='2021-03-15')
# bucket_obj2 = bucket.objects.filter(Prefix='2021-03-16')

objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0],'%Y-%m-%d').date() >= arg_date_dt]

# iterate over bucket object
#objects = [obj for obj in bucket_obj1] + [obj for obj in bucket_obj2]

In [69]:
objects

[s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pd

In [70]:
# just getting the columns from the first dataset
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data_init = StringIO(csv_obj_init)
df_init = pd.read_csv(data_init, delimiter=',')

In [71]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [ ]:
df_all = pd.DataFrame(columns=df_init.columns)

for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    #df_all = df_all.append(df, ignore_index=True)
    df_all = pd.concat([df_all, df], ignore_index=True)

In [ ]:
df_all

In [ ]:
# create list of columns we want to use - dont want to use all above
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']
df_all = df_all.loc[:,columns]

In [ ]:
df_all

In [ ]:
# cleanup if necessary - dont want empty fields
df_all.dropna(inplace=True)

In [ ]:
df_all.shape

## Get Opening Price per ISIN and day.

In [ ]:
# groupby ISIN and Date - based on the StartPrice - then we transform by first 
# transform('first') --> It returns a series / dataframe with the same shape as the source group chunk, 
# in which all values in every individual column are replaced with the first non-NaN value 
# in this column, or with NaN if there is none
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')

In [ ]:
df_all[df_all['ISIN'] == 'AT0000A0E9W5']

## Get Closing Price per ISIN and day

In [ ]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')

In [ ]:
df_all[df_all['ISIN'] == 'AT0000A0E9W5']

## Aggregations

In [ ]:
#as_index=False, needed otherwise ISIN and Date will be the index
#min or max doesnt matter as they are the same in this case for the first 2 columns
#this is to compress/aggregate into one day as the above dataframe is over multiple time periods witihn a day
df_all = df_all.groupby(['ISIN','Date'], as_index=False).agg(opening_price_eur=('opening_price','min'), 
                                                            closing_price_eur=('closing_price','min'), 
                                                            minimum_price_eur=('MinPrice','min'),
                                                            maximum_price_eur=('MaxPrice','max'),
                                                            daily_traded_volume=('TradedVolume','sum'))

In [ ]:
df_all

## Percent Change Prev Closing

In [ ]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [ ]:
df_all

In [ ]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [ ]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [ ]:
df_all = df_all.round(decimals=2)

In [ ]:
df_all = df_all[df_all.Date >= arg_date]

In [ ]:
df_all